# Data Load/Save

In [ ]:
df = pd.read_pickle('../input/a.pkl')
df = pd.read_csv('../input/a.csv', encoding='cp949')
df = pd.read_excel('./files/a.xlsx',
                   header=1, 
                   skipfooter=2,
                   usecols='A:C')

In [ ]:
df.to_excel('./files/a.xlsx', index=False)

# 여러 파일 불러와서 전처리 하는 함수

In [ ]:
def create_kto_data(yy, mm):  
    #1. 불러올 Excel 파일 경로를 지정해주기
    file_path = './files/kto_{}{}.xlsx'.format(yy, mm)  
    
    # 2. Excel 파일 불러오기 
    df = pd.read_excel(file_path, header=1, skipfooter=4, usecols='A:G')
    
    # 3. "기준년월" 컬럼 추가하기 
    df['기준년월'] = '{}-{}'.format(yy, mm) 
    
    # 4. "국적" 컬럼에서 대륙 제거하고 국가만 남기기 
    ignore_list = ['아시아주', '미주', '구주', '대양주', '아프리카주', '기타대륙', '교포소계']
    condition = (df['국적'].isin(ignore_list) == False)    # 대륙 미포함 조건 
    df_country = df[condition].reset_index(drop=True) 
    
    # 5. "대륙" 컬럼 추가하기 
    continents = ['아시아']*25 + ['아메리카']*5 + ['유럽']*23 + ['대양주']*3 + ['아프리카']*2 + ['기타대륙'] + ['교포']
    df_country['대륙'] = continents   
                       
    # 6. 국가별 "관광객비율(%)" 컬럼 추가하기 
    df_country['관광객비율(%)'] = round(df_country.관광 / df_country.계 * 100, 1) 
                       
    # 7. "전체비율(%)" 컬럼 추가하기 
    tourist_sum = sum(df_country['관광'])
    df_country['전체비율(%)'] = round(df_country['관광'] / tourist_sum * 100, 1)
    
    # 8. 결과 출력하기 
    return(df_country)

# 함수를 활용해서 2018년 12월 데이터 불러오기  
kto_test = create_kto_data(2018, 12)

In [ ]:
df = pd.DataFrame()

# 여러 파일 불러와서 합치기
for yy in range(2010, 2020):
    for mm in range(1, 13): 
        try:
            # zfill() 함수 (0을 포함한 두 자리로 출력)
            temp = create_kto_data(str(yy), str(mm).zfill(2))
            df = df.append(temp, ignore_index=True)
        except:
            pass

# DataFrame

In [ ]:
# 데이터 프레임 크기
df.shape
# 데이터 정보
df.info()
# 기초통계량
df.describe()

In [ ]:
df = df.fillna(0)
df.drop('a', axis=1, inplace=True)
df = df.drop(columns=['item_id','author','title_clean','publisher'])

In [ ]:
df.rename(columns={'a':'b'}, inplace=True)

In [ ]:
df2 = df2.merge(df1, 
                on='id', 
                how='left')

df2 = pd.merge(left=df1, 
               right=df2,
               how='left',
               left_on='국적코드',
               right_on='국적코드')

In [ ]:
df2 = df1.groupby(['a','b'])['c'].agg([('column명', 'nunique')]).reset_index()

In [ ]:
df2 = df1.pivot_table(values='입국객수'
                      index='국적명',
                      columns='기준년월',
                      aggfunc='mean' )

df2 = pd.pivot_table(df1, index=['month','id'], columns='c', values='colume명',
                          aggfunc=lambda x: np.where(len(x) >=1, 1, 0), fill_value=0). \
                          reset_index(). \
                          query('month < 12'). \
                          drop(columns=['month','id']).values

In [ ]:
df2 = df1.append(df2, ignore_index=True)

# 필터링

In [2]:
# 조건에 따른 bool 출력
condition = (sample_1['성별'] == '남성')

condition = (sample_1['입국객수'] >= 150000) 

conditions = (sample_1['성별'] == '남성') \
    & (sample_1['입국객수'] >= 150000)

conditions = (sample_1['국적코드'] == 'A01') \
                | (sample_1['국적코드'] == 'A18')

conditions = (sample_1['국적코드'].isin(['A01', 'A18'])) 

# 각 컬럼에서 0인 부분 필터링
condition = (kto_201901['관광'] == 0) \
                | (kto_201901['상용'] == 0) \
                | (kto_201901['공용'] == 0) \
                | (kto_201901['유학/연수'] == 0) 
kto_201901[condition]

# 조건에 맞는 row 출력
sample_1[condition]

# 추가 조건 설정
conditions = (sample_1['국적코드'].isin(['A01', 'A18'])) 
sample_1[conditions == False]

# 정렬

In [4]:
df.sort_values(by='비율(%)', ascending=False)

# lambda 예시

In [ ]:
df['a'] = df['a'].apply(lambda x: x+i)
df['b'] = df['a'].apply(lambda x: 1 if x>0 else 0)

df['b'] = df['a'].apply(lambda x: func(x))
df2['a'] = df1.apply(lambda x: func(x), axis=1)

df['year'] = df['date'].apply(lambda x: x.year)
df['month'] = df['date'].apply(lambda x: x.month)

df2 = df1.groupby('id')['month'].agg([('sales_cnt',lambda x: x.nunique())])
df['age'] = df[['month','date']].apply(lambda x: (2014-x['date'].year)*12 + (x['month'] - x['date'].month) 
                                       if not pd.isnull(x['date']) 
                                       else np.NaN, axis=1).reset_index().sort_values('id', ascending=False)

# debugging

In [ ]:
# 파일 앞 부분에 작성
_debug = False # debugging시 True로 활성화
if _debug == True:
    from ipdb import set_trace

# 이후 원하는 위치 바로 위에 작성
if _debug == True:
    set_trace():
# 해당 부분에서 debugging 시작
# N, C, S, R 등으로 조작
# exit로 종료